In [1]:
import numpy as np 
import tensorflow as tf 
from tensorflow import keras 
import pandas as pd 

In [2]:
df=pd.read_csv('Twitter_Data.csv') 

In [3]:
df.head() 

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [4]:
df.shape 

(162980, 2)

In [5]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [6]:
df.dropna(axis=0,inplace=True) 

In [7]:
df.isnull().sum() 

clean_text    0
category      0
dtype: int64

In [8]:
df.shape 

(162969, 2)

In [9]:
df['category']=df['category'].map({0:0,1:1,-1:2}) 

In [10]:
df['category'].value_counts() 

category
1    72249
0    55211
2    35509
Name: count, dtype: int64

In [11]:
X=df['clean_text']
y=df['category'] 

In [12]:
X=list(X)
y=list(y) 

In [13]:
len(X) 

162969

In [14]:
len(y) 

162969

In [15]:
from keras.preprocessing.text import Tokenizer 
tokenizer=Tokenizer() 

In [16]:
tokenizer.fit_on_texts(X) 

In [17]:
vocab_size=len(tokenizer.word_index)+1 
vocab_size 

113679

In [18]:
seq=tokenizer.texts_to_sequences(X) 

In [19]:
max_len=max([len(i) for i in seq]) 

In [20]:
max_len 

52

In [21]:
X_final=keras.preprocessing.sequence.pad_sequences(seq,maxlen=max_len,padding='pre') 

In [22]:
X_final[0] 

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,    42,     1,   307, 42492,    66,  1726, 42493,  1119,
          40,  2378,     2,  1211,   205, 17321,     2,   215,    32,
         155,   100,    49,    69,  1068,   215,    50,     3,     6,
         546,     3,    50,  4179,  5582,     3,  2806])

In [23]:
y=keras.utils.to_categorical(y,num_classes=3) 

In [24]:
y[:5] 

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [25]:
model=keras.Sequential([
    keras.layers.Embedding(vocab_size,100,input_length=max_len),
    keras.layers.LSTM(100,return_sequences=True),
    keras.layers.Dropout(0.2),
    keras.layers.LSTM(100),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(3,activation='softmax')
]) 

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
) 

In [26]:
model.fit(X_final,y,epochs=5) 

Epoch 1/5
5093/5093 [==============================] - 123s 23ms/step - loss: 0.2109 - accuracy: 0.9284
Epoch 2/5
5093/5093 [==============================] - 120s 24ms/step - loss: 0.0712 - accuracy: 0.9774
Epoch 3/5
5093/5093 [==============================] - 123s 24ms/step - loss: 0.0403 - accuracy: 0.9870
Epoch 4/5
5093/5093 [==============================] - 119s 23ms/step - loss: 0.0240 - accuracy: 0.9926
Epoch 5/5
5093/5093 [==============================] - 119s 23ms/step - loss: 0.0136 - accuracy: 0.9958


In [36]:
def predict_sentiment(x):
    x=[x]
    x_seq=tokenizer.texts_to_sequences(x)
    x_final=keras.preprocessing.sequence.pad_sequences(x_seq,maxlen=max_len,padding='pre') 
    pred=model.predict(x_final)
    val=np.argmax(pred)
    if val==1:
        print("The sentiment is Positive")
    elif val==0:
        print("The sentiment is Neutral")
    else:
        print("The sentiment is Negative")

In [37]:
predict_sentiment("This is a great place to work") 

1/1 [==============================] - 0s 34ms/step
The sentiment is Positive


In [38]:
predict_sentiment("He might be a perfect fit for the squad") 

1/1 [==============================] - 0s 30ms/step
The sentiment is Positive


In [39]:
predict_sentiment("He is always the worst player in team")

1/1 [==============================] - 0s 75ms/step
The sentiment is Negative


In [40]:
predict_sentiment("I dont know anything about him") 

1/1 [==============================] - 0s 34ms/step
The sentiment is Neutral


In [41]:
predict_sentiment("He is not good at all for this job") 

1/1 [==============================] - 0s 30ms/step
The sentiment is Negative


In [44]:
predict_sentiment("He is a person always confused while making decisions") 

1/1 [==============================] - 0s 30ms/step
The sentiment is Negative


In [42]:
model.save("sentiment_model.keras") 